# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

## imports for Python, Pandas

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

****
## JSON exercise

Using data in file 'data/world_bank_projects.json',
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [2]:
# load json as string
data = json.load((open('Downloads/data_wrangling_json/data_wrangling_json/data/world_bank_projects.json')))

In [3]:
#Check the number of elements present in the list#
len(data)

500

****
#  To find top 10 countries
We need to check the Key 'countryshortname' in all the 500 occurence of dictionaries in the list. 
I have used List comprehension to get all the elements of dictionary and then specifocally picked the Key 'countryshortname'. Then I used the value_counts on the Typecasted Series to count the countries occurence. Then I selected the first 10 occureneces only as per the reuqest in the Question

In [4]:
country_top_10 = pd.Series([dic['countryshortname'] for dic in data]).value_counts()[:10]
country_top_10

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Mozambique            11
Africa                11
dtype: int64

# To find the top 10 major project themes 
To find the top 10 major project theme we need to use column 'mjtheme_namecode'. This column is a list of dictionary 
which can be normalized using the function json_normalize. We can use count_values to check the top 10 project themes 

In [5]:
themes = json_normalize(data, 'mjtheme_namecode')
Themes_top_10 = themes['name'].value_counts()[:10]
#Print the top 10 major project themes
Themes_top_10

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

# To fill empty project themes 
In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [6]:
#Check whether the themes name is SPACE os not
themes[themes['name']==''].empty
# As we can see in the output that its false. It means that there are some theme names missing in the JSON file provided. 

False

In [7]:
#looking at the data again we can see that there is no data having code and name as empty together. 
#And there are many records where name is empty only
name_check = themes[themes['name']==''].empty
code_check = themes[(themes['name']=='') & (themes['code']=='')].empty
 
print("Is name empty? ", name_check)
print("Is name & code empty? ",code_check) 

Is name empty?  False
Is name & code empty?  True


In [8]:
# Every code has corresponding theme name. Code is available but corresponding name is missing for it. 
#So using the code we will fill the missing values of theme name

#Create the dictionary for all the unique themes with their code
Theme_code_dic = dict([(nm,cd) for nm, cd in zip(themes['name'], themes['code'])])
# drop the space Theme key from dictionary
Theme_code_dic.pop('',None)

#reverse the Key:Value dictionary. 
code_theme_dic = {i[1]:i[0] for i in Theme_code_dic.items()}
code_theme_dic

{'8': 'Human development',
 '1': 'Economic management',
 '6': 'Social protection and risk management',
 '5': 'Trade and integration',
 '2': 'Public sector governance',
 '11': 'Environment and natural resources management',
 '7': 'Social dev/gender/inclusion',
 '4': 'Financial and private sector development',
 '10': 'Rural development',
 '9': 'Urban development',
 '3': 'Rule of law'}

In [9]:
# fill all the empty values of themes using the dictioary created above
# Saves the data in mod_themes
mod_themes = themes
for i in mod_themes.index:
    if mod_themes.iloc[i]['name'] == '':
        mod_themes.iloc[i]['name'] = code_theme_dic[mod_themes.iloc[i]['code']]

mod_themes.head()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [10]:
mod_themes_top_10 = mod_themes['name'].value_counts()[:10]
#Print the top 10 major project themes
mod_themes_top_10

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

# Conclusion

There are few things to be noticed both the dataframe (Themes and mod_themes) that are:
    1. There is no empty theme in the mod_theme dataframe
    2. Though most of the rankings for project theme remains same, but there is a significant change 
        in the number of projects in each category
    3. Point 2 also suggest that missing data is randomly missing from all the categories.
    4. In top 10 after imputing missing values, Economic Management appeared in the list now.
    5. Its always good to impute the data more logically rather simply dropping the data.
    